## Data preprocess

In [46]:
import numpy as np
import pandas as pd


def one_hot_encoding(arr):
    s = set()
    _sd = dict()
    for e in arr:
        s.add(e)

    sl = list(s)
    for i in range(len(sl)):
        encode = np.zeros(len(sl))
        encode[i] = 1
        _sd[sl[i]] = encode

    encoded_data = []
    for k in range(len(arr)):
        encoded_data.append(_sd[arr[k]])

    return np.array(encoded_data)


def data_preprocess(filepath):
    df = pd.read_excel(filepath)
    data = df.to_numpy()

    ip = data[:, 2]

    encoded_ip = one_hot_encoding(ip)

    # print(encoded_ip)

    data[:, 2] = encoded_ip.tolist()

    # print(data)
    expanded_data = []
    for row in data:
        expanded_data.append(np.concatenate([row[:2], row[2], row[3:]]).tolist())

    expanded_data = np.array(expanded_data)

    for i in range(len(expanded_data)):
        for j in range(len(expanded_data[i])):
            expanded_data[i][j] = float(expanded_data[i][j])

    return expanded_data[:]


filename = "processTime_reuqestNumber_v3"

# Data preprocessing
file_path = f"D:\\model_fit\\training\\training_data\\{filename}.xlsx"
dataset = data_preprocess(file_path)

print(dataset)

[[5.50447092e+01 2.80807000e+05 0.00000000e+00 0.00000000e+00
  1.00000000e+00 3.54374146e+00]
 [9.84394312e+00 1.74622000e+05 0.00000000e+00 1.00000000e+00
  0.00000000e+00 1.81988001e+00]
 [1.55435622e+01 3.87111000e+05 1.00000000e+00 0.00000000e+00
  0.00000000e+00 5.69155121e+00]
 [1.71737144e+01 3.96214000e+05 0.00000000e+00 0.00000000e+00
  1.00000000e+00 5.91288614e+00]
 [2.43334904e+01 7.64103000e+05 0.00000000e+00 1.00000000e+00
  0.00000000e+00 1.53142524e+01]
 [8.01431279e+01 7.32257000e+05 0.00000000e+00 1.00000000e+00
  0.00000000e+00 1.43273647e+01]
 [3.31535378e+01 7.83373000e+05 1.00000000e+00 0.00000000e+00
  0.00000000e+00 1.59812455e+01]
 [8.02251434e+00 4.82275000e+05 0.00000000e+00 0.00000000e+00
  1.00000000e+00 7.87945366e+00]
 [1.12581830e+01 6.11868000e+05 0.00000000e+00 1.00000000e+00
  0.00000000e+00 1.11101882e+01]
 [7.05524948e+01 7.62759000e+05 1.00000000e+00 0.00000000e+00
  0.00000000e+00 1.51405113e+01]
 [2.11190414e+00 1.83562000e+05 1.00000000e+00 0.0

## XGBOOST - 1

In [47]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X, y = dataset[:, 1:-1], dataset[:, -1]


train_size = int(len(X) * 0.7)

# 拆分数据集为训练集和测试集
# split dataset to train dataset and test dataset

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# time series
# X_train = X[:train_size]
# X_test = X[train_size:]
# y_train = y[:train_size]
# y_test = y[train_size:]

In [48]:
from xgboost import XGBRegressor

# Create XGBoost regression model
model = XGBRegressor(
    objective="reg:squarederror",   # Loss function to minimize: squared error for regression  损失函数为平方损失函数, 
    n_estimators=100,  # Number of boosting rounds (number of trees)  迭代次数，即基础学习器的数量
    learning_rate=0.1,  # Boosting learning rate (controls the step size)  学习率，控制每次更新的步长
    max_depth=5,  # Maximum depth of a tree  树的最大深度
    min_child_weight=1,  # Minimum sum of instance weight (hessian) needed in a child  叶子节点最小权重
    subsample=0.8,  # Subsample ratio of the training instance (randomly sampled)  每棵树随机采样的比例
    colsample_bytree=0.8,  # Subsample ratio of columns when constructing each tree  每棵树随机选择的特征比例
    alpha=0.01,  # L1 regularization term on weights  L1 正则化项参数
    reg_lambda=0.01,  # L2 regularization term on weights  L2 正则化项参数
)

# model fit
model.fit(X_train, y_train)

XGBRegressor(alpha=0.01, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, ...)

In [49]:
from sklearn.metrics import mean_absolute_error

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mea = mean_absolute_error(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mea}")


print(f"    real    |    pred   \n")
for i in range(len(y_test)):
    print(f"    {y_test[i]}     |     {y_pred[i]}      ")

Mean Squared Error: 0.5474930431287381
Mean Absolute Error: 0.47088616266846656
    real    |    pred   

    13.60307049751282     |     13.106794357299805      
    20.29998636245728     |     18.233505249023438      
    12.38283538818359     |     11.838896751403809      
    17.12697076797485     |     16.56221580505371      
    3.612605333328247     |     3.1008667945861816      
    15.84868288040161     |     15.955033302307129      
    0.2680389881134033     |     0.12532925605773926      
    0.8576817512512207     |     0.8540836572647095      
    0.1278862953186035     |     0.07089265435934067      
    17.10650610923767     |     16.890485763549805      


## XGBRegressor

In [50]:
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error


# 创建 XGBoost 回归模型
model = XGBRegressor(
    objective="reg:squarederror",
    n_estimators=100,
    learning_rate=0.1,
    tree_method="hist",
    device="cuda",
)

# Define the parameter grid for GridSearchCV
param_grid = {
    "max_depth": [3],  # Range of maximum depth of trees to explore  扩展最大深度的范围
    "learning_rate": [0.1],  # Range of learning rates to explore  扩展学习率的范围
    "n_estimators": [100],  # Range of number of trees (boosting rounds) to explore  扩展树的数量范围
    "gamma": [0],   # Range of gamma parameter to explore (controls tree split) 扩展 gamma 参数范围
    "subsample": [1],  # Range of subsample ratios to explore  扩展子样本比例范围
    "colsample_bytree": [0.8, 0.7],  # Range of column subsample ratios for each tree to explore  扩展列采样比例范围
    "reg_alpha": [0, 0.5],  # Range of L1 regularization parameters to explore  扩展 L1 正则化参数范围
    "reg_lambda": [0, 0.5],  # Range of L2 regularization parameters to explore  扩展 L2 正则化参数范围
}


grid_search = GridSearchCV(
    estimator=model, param_grid=param_grid, cv=5, scoring="neg_mean_squared_error"
)
grid_search.fit(X_train, y_train)

print("Best parameters found: ", grid_search.best_params_)

# 使用最佳参数重新训练模型
# Re-train the model using the best parameters
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train, verbose=True)


Best parameters found:  {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'reg_alpha': 0, 'reg_lambda': 0, 'subsample': 1}


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device='cuda', early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [76]:
preds = best_model.predict(X_test)

rmse = mean_squared_error(y_test, preds, squared=False)
print(f"RMSE: {rmse}")

mae = mean_absolute_error(y_test, preds)
print(f"MAE: {mae}")


avg = np.mean(y_test)

print(f"AVG: {avg}")

print(f"\t\treal\t\t|\t\tpred\t\t|\t\tdiff\t\t|\t\tdiff_per\t|\n")
a = 0
for i in range(len(y_test)):
    if (abs(y_test[i] - y_pred[i]))  <= 1:
        a += 1
    print(
        f"\t{y_test[i]}\t|\t{y_pred[i]}\t|\t{abs(y_test[i] - y_pred[i])}\t|\t{(abs(y_test[i] - y_pred[i])) / y_test[i]}\t|",
    )

print(f"Accuracy in 500ms: {round(100 * a / len(y_test), 3)}%")

RMSE: 0.7573068781327319
MAE: 0.4755382537841797
AVG: 10.12342643737793
		real		|		pred		|		diff		|		diff_per	|

	13.60307049751282	|	13.106794357299805	|	0.49627614021301447	|	0.0364826559050586	|
	20.29998636245728	|	18.233505249023438	|	2.0664811134338414	|	0.10179716757128389	|
	12.38283538818359	|	11.838896751403809	|	0.5439386367797816	|	0.04392682448955422	|
	17.12697076797485	|	16.56221580505371	|	0.564754962921139	|	0.03297459723450661	|
	3.612605333328247	|	3.1008667945861816	|	0.5117385387420654	|	0.1416535966497639	|
	15.84868288040161	|	15.955033302307129	|	0.10635042190551935	|	0.0067103634231354126	|
	0.2680389881134033	|	0.12532925605773926	|	0.14270973205566406	|	0.5324215445675601	|
	0.8576817512512207	|	0.8540836572647095	|	0.0035980939865112305	|	0.004195138792754056	|
	0.1278862953186035	|	0.07089265435934067	|	0.05699364095926282	|	0.4456587065664417	|
	17.10650610923767	|	16.890485763549805	|	0.2160203456878662	|	0.012627964138814602	|
Accuracy in 500ms: 90.0%


d:\model_fit\.venv\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
